In [1]:
import json
import os, sys
import pandas as pd
import pickle
import math
import shutil
import numpy as np
import glob
import librosa
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *

In [1]:
joint_list = {
    "beat_joints" : {
        'Hips':         [6,6],
        'Spine':        [3,9],
        'Spine1':       [3,12],
        'Spine2':       [3,15],
        'Spine3':       [3,18],
        'Neck':         [3,21],
        'Neck1':        [3,24],
        'Head':         [3,27],
        'HeadEnd':      [3,30],

        'RShoulder':    [3,33], 
        'RArm':         [3,36],
        'RArm1':        [3,39],
        'RHand':        [3,42],    
        'RHandM1':      [3,45],
        'RHandM2':      [3,48],
        'RHandM3':      [3,51],
        'RHandM4':      [3,54],

        'RHandR':       [3,57],
        'RHandR1':      [3,60],
        'RHandR2':      [3,63],
        'RHandR3':      [3,66],
        'RHandR4':      [3,69],

        'RHandP':       [3,72],
        'RHandP1':      [3,75],
        'RHandP2':      [3,78],
        'RHandP3':      [3,81],
        'RHandP4':      [3,84],

        'RHandI':       [3,87],
        'RHandI1':      [3,90],
        'RHandI2':      [3,93],
        'RHandI3':      [3,96],
        'RHandI4':      [3,99],

        'RHandT1':      [3,102],
        'RHandT2':      [3,105],
        'RHandT3':      [3,108],
        'RHandT4':      [3,111],

        'LShoulder':    [3,114], 
        'LArm':         [3,117],
        'LArm1':        [3,120],
        'LHand':        [3,123],    
        'LHandM1':      [3,126],
        'LHandM2':      [3,129],
        'LHandM3':      [3,132],
        'LHandM4':      [3,135],

        'LHandR':       [3,138],
        'LHandR1':      [3,141],
        'LHandR2':      [3,144],
        'LHandR3':      [3,147],
        'LHandR4':      [3,150],

        'LHandP':       [3,153],
        'LHandP1':      [3,156],
        'LHandP2':      [3,159],
        'LHandP3':      [3,162],
        'LHandP4':      [3,165],

        'LHandI':       [3,168],
        'LHandI1':      [3,171],
        'LHandI2':      [3,174],
        'LHandI3':      [3,177],
        'LHandI4':      [3,180],

        'LHandT1':      [3,183],
        'LHandT2':      [3,186],
        'LHandT3':      [3,189],
        'LHandT4':      [3,192],

        'RUpLeg':       [3,195],
        'RLeg':         [3,198],
        'RFoot':        [3,201],
        'RFootF':       [3,204],
        'RToeBase':     [3,207],
        'RToeBaseEnd':  [3,210],

        'LUpLeg':       [3,213],
        'LLeg':         [3,216],
        'LFoot':        [3,219],
        'LFootF':       [3,222],
        'LToeBase':     [3,225],
        'LToeBaseEnd':  [3,228],
        },
    
    "beat_141" : {
            'Spine':       3 ,
            'Neck':        3 ,
            'Neck1':       3 ,
            'RShoulder':   3 , 
            'RArm':        3 ,
            'RArm1':       3 ,
            'RHand':       3 ,    
            'RHandM1':     3 ,
            'RHandM2':     3 ,
            'RHandM3':     3 ,
            'RHandR':      3 ,
            'RHandR1':     3 ,
            'RHandR2':     3 ,
            'RHandR3':     3 ,
            'RHandP':      3 ,
            'RHandP1':     3 ,
            'RHandP2':     3 ,
            'RHandP3':     3 ,
            'RHandI':      3 ,
            'RHandI1':     3 ,
            'RHandI2':     3 ,
            'RHandI3':     3 ,
            'RHandT1':     3 ,
            'RHandT2':     3 ,
            'RHandT3':     3 ,
            'LShoulder':   3 , 
            'LArm':        3 ,
            'LArm1':       3 ,
            'LHand':       3 ,    
            'LHandM1':     3 ,
            'LHandM2':     3 ,
            'LHandM3':     3 ,
            'LHandR':      3 ,
            'LHandR1':     3 ,
            'LHandR2':     3 ,
            'LHandR3':     3 ,
            'LHandP':      3 ,
            'LHandP1':     3 ,
            'LHandP2':     3 ,
            'LHandP3':     3 ,
            'LHandI':      3 ,
            'LHandI1':     3 ,
            'LHandI2':     3 ,
            'LHandI3':     3 ,
            'LHandT1':     3 ,
            'LHandT2':     3 ,
            'LHandT3':     3 ,
        },
    
    "beat_27" : {
            'Spine':       3 ,
            'Neck':        3 ,
            'Neck1':       3 ,
            'RShoulder':   3 , 
            'RArm':        3 ,
            'RArm1':       3 ,
            'LShoulder':   3 , 
            'LArm':        3 ,
            'LArm1':       3 ,
        },
}
target_fps = 15
ori_data_path = f"./datasets/beat_rawdata_english"
cache_path = f"./datasets/beat_cache/beat_4english_{target_fps}_141/train"

In [3]:
#define which speakers included in current train
#speakers = list(map(int, sorted(os.listdir(ori_data_path),key=str,)))
speakers = [2, 4, 6, 8]
print(speakers)

[2, 4, 6, 8]


In [ ]:
#transform all .wav to .npm using librosa
import time
audio_load_start_time = time.time()
for file in glob.glob(f"{ori_data_path}/**/*.wav", recursive=True):
    #print(f"making cache for {file} to {file[:-4]}.npy")
    audio, sr = librosa.load(f"{file}", sr=16000)
    #np.save(f"{file[:-4]}.npy", audio)
print(f"{time.time() - audio_load_start_time}")

61.08784103393555


In [5]:
#makedirs for cache data
os.makedirs(f"{cache_path}/")
os.mkdir(f"{cache_path}/bvh_rot/")
os.mkdir(f"{cache_path}/bvh_rot_6d/")
#os.mkdir(f"{cache_path}/bvh_full/")
os.mkdir(f"{cache_path}/bvh_rot_vis/")
os.mkdir(f"{cache_path}/head_rot/")
os.mkdir(f"{cache_path}/wave16k/")
os.mkdir(f"{cache_path}/facial52/")
os.mkdir(f"{cache_path}/text/")
os.mkdir(f"{cache_path}/emo/")
os.mkdir(f"{cache_path}/sem/")

In [8]:
#calculate mean and build cache for data. 
ori_list = joint_list["beat_joints"]
target_list = joint_list["beat_141"]
#wave cache from a = librosa.load(sr=16000) and np.save(a)
reduce_factor_json = int(60/target_fps)
reduce_factor_bvh = int(120/target_fps)
print(f"target_fps: {target_fps}, reduce json {reduce_factor_json}, reduce bvh {reduce_factor_bvh}")

npy_s_v = []
npy_s_k = []
json_s_v = []
bvh_s_v = []

def adjust_angles_to_pi_range(angles):
    adjusted = np.mod(angles, 360)
    adjusted[adjusted > 180] -= 360
    return adjusted

for speaker in tqdm(speakers):
    all_data = os.listdir(f"{ori_data_path}/{speaker}")
    npy_all = []
    json_all = []
    bvh_all = []
    for ii, file in tqdm(enumerate(all_data)):
        if not file.startswith(str(speaker)):
            print(f"<!>{file} belongs an incorrect folder in {speaker}.")
        if not file.endswith(".npy"):
            continue
        file = file[:-4]
        try: shutil.copy(f"{ori_data_path}/{speaker}/{file}.txt", f"{cache_path}/sem/{file}.txt")
        except: print(f"{file}.txt")
        try: shutil.copy(f"{ori_data_path}/{speaker}/{file}.csv", f"{cache_path}/emo/{file}.csv")
        except: print(f"{file}.csv")
        #audio npy
        shutil.copy(f"{ori_data_path}/{speaker}/{file}.npy", f"{cache_path}/wave16k/{file}.npy")
        npy_all.extend(list(np.load(f"{ori_data_path}/{speaker}/{file}.npy")))
        #text TextGrid
        try:shutil.copy(f"{ori_data_path}/{speaker}/{file}.TextGrid", f"{cache_path}/text/{file}.TextGrid")
        except: print(f"{file}.TextGrid")
        #facial json
        with open(f"{ori_data_path}/{speaker}/{file}.json", "r", encoding='utf-8') as json_file_raw:
            json_file = json.load(json_file_raw)
            with open(f"{cache_path}/facial52/{file}.json", "w") as reduced_json:
                counter = 0
                new_frames_list = []
                for json_data in json_file["frames"]:
                    if counter % reduce_factor_json == 0:
                        json_all.append(json_data["weights"])
                        new_frames_list.append(json_data)
                    counter += 1
                json.dump({"names":json_file["names"], "frames": new_frames_list}, reduced_json)
        #body bvh
        with open(f"{ori_data_path}/{speaker}/{file}.bvh", "r") as bvh_file:
            with open(f"{cache_path}/bvh_rot/{file}.bvh", "w") as reduced_raw_bvh: # This is trainable
                with open(f"{cache_path}/bvh_rot_vis/{file}.bvh", "w") as reduced_trainable_bvh: # This is raw (blender previewable)
                    print(f"file: {file}")
                    for i, line_data in enumerate(bvh_file.readlines()):
                        # angle range check
                        data = np.fromstring(line_data, dtype=float, sep=' ')
                        data = adjust_angles_to_pi_range(data)
                        if i < 431:
                            reduced_trainable_bvh.write(line_data)
                            continue
                        if i % reduce_factor_bvh != 0:
                            continue
                        # one line(i) = one frame
                        bvh_all.append(data)
                        trainable_rotation = np.zeros_like(data)
                        for k, eular in target_list.items():
                            trainable_rotation[ori_list[k][1]-eular:ori_list[k][1]] = data[ori_list[k][1]-eular:ori_list[k][1]]
                        trainable_line_data = np.array2string(trainable_rotation, max_line_width=np.inf, precision=6, suppress_small=False, separator=' ')
                        reduced_trainable_bvh.write(trainable_line_data[1:-2]+"\n")
                        
                        data_rotation = np.zeros((1))
                        for k, eular in target_list.items():
                            data_rotation = np.concatenate((data_rotation, data[ori_list[k][1]-eular:ori_list[k][1]]))                             
                        raw_line_data = np.array2string(data_rotation[1:], max_line_width=np.inf, precision=6, suppress_small=False, separator=' ')
                        reduced_raw_bvh.write(raw_line_data[1:-2] + '\n')
    npy_all = np.array(npy_all)
    npy_mean = np.mean(npy_all, axis=0)
    npy_std = np.std(npy_all, axis=0)
    npy_s_v.append([npy_mean, npy_std])
    npy_s_k.append([len(npy_all)/16000/60])
    np.save(f"{cache_path}/wave16k/npy_mean_{speaker}.npy", npy_mean)
    np.save(f"{cache_path}/wave16k/npy_std_{speaker}.npy", npy_std)  

    cjson_all = np.array(json_all)
    json_mean = np.mean(json_all, axis=0)
    json_std = np.std(json_all, axis=0)
    json_s_v.append([json_mean, json_std])
    np.save(f"{cache_path}/facial52/json_mean_{speaker}.npy", json_mean)
    np.save(f"{cache_path}/facial52/json_std_{speaker}.npy", json_std)

    bvh_all = np.array(bvh_all)
    bvh_mean = np.mean(bvh_all, axis=0)
    bvh_std = np.std(bvh_all, axis=0)
    bvh_s_v.append([bvh_mean, bvh_std])
    # number of joints reduced version 
    data_rotation = np.zeros((1))
    for k, eular in target_list.items():
        data_rotation = np.concatenate((data_rotation, bvh_mean[ori_list[k][1]-eular:ori_list[k][1]]))
    new_npy_mean = data_rotation[1:]
    data_rotation = np.zeros((1))
    for k, eular in target_list.items():
        data_rotation = np.concatenate((data_rotation, bvh_std[ori_list[k][1]-eular:ori_list[k][1]]))
    new_npy_std = data_rotation[1:]
    np.save(f"{cache_path}/bvh_rot/bvh_mean_{speaker}.npy", new_npy_mean)
    np.save(f"{cache_path}/bvh_rot/bvh_std_{speaker}.npy", new_npy_std)

target_fps: 15, reduce json 4, reduce bvh 8


  0%|          | 0/4 [00:00<?, ?it/s]

file: 2_scott_0_91_91


/tmp/ipykernel_2504227/1142981413.py:59: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  data = np.fromstring(line_data, dtype=float, sep=' ')


file: 2_scott_0_6_6


...

file: 8_catherine_0_21_21


file: 8_catherine_0_99_99


file: 8_catherine_0_67_67


file: 8_catherine_0_92_92


910it [08:02,  1.89it/s]
100%|██████████| 4/4 [34:45<00:00, 521.38s/it]


In [7]:
%pwd

'/mnt/data/izumi/BEAT_2022/dataloaders'

In [4]:
import torch
import sys
sys.path.append('..')
from utils import rotation_tools
update_rot6d = False
update_axis_angle = True
update_head_pose_rot6d = False
def insert_new_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
def convert_representations_from_bvh(path):
    all_data = os.listdir(f"{path}/bvh_rot")
    bvh_rot6d_all = []
    bvh_axis_angle_all = []
    head_rot_all = []
    print(f"#create in {path}")
    insert_new_dirs(f"{path}/bvh_axis_angle")
    for ii, file in tqdm(enumerate(all_data)):
        #if not file.startswith("2"):
        #    continue
        if not file.endswith("bvh"):
            continue
        with open(f"{path}/bvh_rot/{file}", "r") as bvh_file:
            pose_rot6d_file = open(f"{path}/bvh_rot_6d/{file[:-3]}6d", "w") if update_rot6d else None
            pose_axis_angle_file = open(f"{path}/bvh_axis_angle/{file[:-3]}axisangle", "w") if update_axis_angle else None
            head_rot_file = open(f"{path}/head_rot/{file[:-3]}6d", "w") if update_head_pose_rot6d else None
            try:
                #prev_data = None
                #prev_vel = None
                for i, line_data in enumerate(bvh_file.readlines()):
                    data = np.fromstring(line_data, dtype=float, sep=' ')
                    data = np.deg2rad(data)
                    data = rotation_tools.euler_angles_to_matrix(torch.from_numpy(data.reshape([-1, 3])), "XYZ")
                    # repeat conversion stabilize (according to EMAGE emage_trainer)
                    #data = rotation_tools.matrix_to_rotation_6d(data)
                    #data = rotation_tools.rotation_6d_to_matrix(data)
                    if head_rot_file is not None:
                        spine1 = data[0, :]
                        spine2 = data[1, :]
                        neck = data[2, :]
                        head_rot = torch.matmul(torch.matmul(spine1, spine2), neck)
                        head_rot = rotation_tools.matrix_to_rotation_6d(head_rot).numpy().reshape(-1)
                        head_rot_all.append(head_rot)
                        trainable_line_data = np.array2string(head_rot, max_line_width=np.inf, precision=6, suppress_small=False, separator=' ')
                        head_rot_file.write(trainable_line_data[1:-1] + '\n')
                    if pose_axis_angle_file is not None:
                        data_axis_angle = rotation_tools.matrix_to_axis_angle_v2(data).numpy().reshape(-1)
                        bvh_axis_angle_all.append(data_axis_angle)
                        trainable_line_data = np.array2string(data_axis_angle, max_line_width=np.inf, precision=6, suppress_small=False, separator=' ')
                        pose_axis_angle_file.write(trainable_line_data[1:-1] + '\n')
                    if pose_rot6d_file is not None:
                        data = rotation_tools.matrix_to_rotation_6d(data).numpy().reshape(-1)
                        bvh_rot6d_all.append(data)
                        trainable_line_data = np.array2string(data, max_line_width=np.inf, precision=6, suppress_small=False, separator=' ')
                        pose_rot6d_file.write(trainable_line_data[1:-1] + '\n')
            finally:
                if head_rot_file is not None:
                    head_rot_file.close()
                if pose_axis_angle_file is not None:
                    pose_axis_angle_file.close()
                if pose_rot6d_file is not None:
                    pose_rot6d_file.close()
    if update_rot6d:
        bvh_rot6d_all = np.array(bvh_rot6d_all)
        mean = np.mean(bvh_rot6d_all, axis=0)
        std = np.std(bvh_rot6d_all, axis=0)
        np.save(f"{path}/bvh_rot_6d/bvh_rot_6d_mean.npy", mean)
        np.save(f"{path}/bvh_rot_6d/bvh_rot_6d_std.npy", std)
    if update_axis_angle:
        bvh_axis_angle_all = np.array(bvh_axis_angle_all)
        mean = np.mean(bvh_axis_angle_all, axis=0)
        std = np.std(bvh_axis_angle_all, axis=0)
        np.save(f"{path}/bvh_axis_angle/bvh_axis_angle_mean.npy", mean)
        np.save(f"{path}/bvh_axis_angle/bvh_axis_angle_std.npy", std)
    if update_head_pose_rot6d:
        head_rot_all = np.array(head_rot_all)
        mean = np.mean(head_rot_all, axis=0)
        std = np.std(head_rot_all, axis=0)
        np.save(f"{path}/head_rot/head_rot_mean.npy", mean)
        np.save(f"{path}/head_rot/head_rot_std.npy", std)
    #np.save(f"{path}/bvh_rot/eular_ang_vel_mean_2.npy", np.mean(vel_all, axis=0))
    #np.save(f"{path}/bvh_rot/eular_ang_vel_std_2.npy", np.std(vel_all, axis=0))
    #np.save(f"{path}/bvh_rot/eular_ang_acc_mean_2.npy", np.mean(acc_all, axis=0))
    #np.save(f"{path}/bvh_rot/eular_ang_acc_std_2.npy", np.std(acc_all, axis=0))

convert_representations_from_bvh(cache_path)
convert_representations_from_bvh(cache_path.replace("train", "val"))
convert_representations_from_bvh(cache_path.replace("train", "test"))

#create in ./datasets/beat_cache/beat_4english_15_141/train


402it [16:23,  2.45s/it]


#create in ./datasets/beat_cache/beat_4english_15_141/val


64it [02:12,  2.06s/it]


#create in ./datasets/beat_cache/beat_4english_15_141/test


64it [02:20,  2.20s/it]


In [9]:
# calculate global mean and std for average all speakers
npy_s_v = []
json_s_v = []
bvh_s_v = []

npy_path = f"{cache_path}/wave16k"
bvh_path = f"{cache_path}/bvh_rot"
json_path = f"{cache_path}/facial52" 
        
for i in speakers:
    npy_s_v.append([np.load(f"{npy_path}/npy_mean_{i}.npy"), np.load(f"{npy_path}/npy_std_{i}.npy")])
    json_s_v.append([np.load(f"{json_path}/json_mean_{i}.npy"), np.load(f"{json_path}/json_std_{i}.npy")])
    bvh_s_v.append([np.load(f"{bvh_path}/bvh_mean_{i}.npy"), np.load(f"{bvh_path}/bvh_std_{i}.npy")])

all_length = 0
new_m = new_s = 0
#new_m = np.zeros_like(npy_s_v[0][0])
#new_s = np.zeros_like(npy_s_v[0][0])
for i, (m, s) in enumerate(npy_s_v):
    all_length += npy_s_k[i][0]
    new_m += npy_s_k[i][0] * m
new_m /= all_length
for i, (m, s) in enumerate(npy_s_v):
    new_s += ((s**2) + (m-new_m)**2) * npy_s_k[i][0]
new_s /= all_length
new_s = np.sqrt(new_s)
print(f"audio mean: {new_m}, std: {new_s}") 
np.save(f"{npy_path}/npy_mean.npy", new_m)
np.save(f"{npy_path}/npy_std.npy", new_s)  

all_length = 0
new_m = new_s = 0
#new_m = np.zeros_like(json_s_v[0][0])
#new_s = np.zeros_like(json_s_v[0][0])
for i, (m, s) in enumerate(json_s_v):
    all_length += npy_s_k[i][0]
    new_m += npy_s_k[i][0] * m
new_m /= all_length
for i, (m, s) in enumerate(json_s_v):
    new_s += ((s**2) + (m-new_m)**2) * npy_s_k[i][0]
new_s /= all_length
new_s = np.sqrt(new_s)
np.save(f"{json_path}/json_mean.npy", new_m)
np.save(f"{json_path}/json_std.npy", new_s)

all_length = 0
new_m = new_s = 0
#new_m = np.zeros_like(bvh_s_v[0][0])
#new_s = np.zeros_like(bvh_s_v[0][0])
for i, (m, s) in enumerate(bvh_s_v):
    all_length += npy_s_k[i][0]
    new_m += npy_s_k[i][0] * m
new_m /= all_length
for i, (m, s) in enumerate(bvh_s_v):
    new_s += ((s**2) + (m-new_m)**2) * npy_s_k[i][0]
new_s /= all_length
new_s = np.sqrt(new_s)

print(new_m.shape)
print(new_s.shape)
print(f"bvh mean: {new_m}, std: {new_s}") 
np.save(f"{bvh_path}/bvh_mean.npy", new_m)
np.save(f"{bvh_path}/bvh_std.npy", new_s)

audio mean: -1.753280171839557e-05, std: 0.08794526423877846
(141,)
(141,)
bvh mean: [ 1.59045798e+01  1.92844329e-01 -6.30230118e-01  7.28802744e+00
 -2.65845237e+00  1.10275323e+00 -3.65125947e+00 -5.25760040e+00
 -7.22091144e+00 -5.37784132e-06 -1.49665291e+01  4.33142915e+00
 -1.32733595e+01  2.40613776e+01  6.46167496e+01 -5.28339298e-01
  8.07433555e+01 -8.70368683e-01  1.63797944e+00 -5.26727977e-01
  1.58542881e+00  8.90394390e-01 -3.74767772e+00  2.50531375e+01
  4.12687317e-07  3.51824007e-07  2.37111413e+01  4.67951454e-07
  2.50737513e-07  1.78241289e+01 -1.08300418e+01 -1.73534735e+00
 -3.72940357e-01  3.64240248e-01 -7.38525387e+00  2.20167394e+01
 -2.30022650e+00 -8.90069862e-01  2.83422697e+01 -1.04638173e+00
 -1.65196600e-01  1.17959571e+01 -5.20084963e+00 -2.45326875e+00
 -1.16790017e+00 -2.04586052e+00 -7.98968134e+00  9.79647160e+00
 -4.80577663e+00 -1.52134931e+00  2.29946430e+01 -4.42791389e+00
 -1.01392826e+00  2.05283097e+01 -4.72301116e+00  6.83880062e+00
  3.3

In [10]:
default_path = "./datasets/beat_cache/beat_4english_15_141/train"
speaker_names = [
    "wayne", "scott", "solomon", "lawrence", "stewart", "carla", "sophie", "catherine", "miranda", "kieks", \
    "nidal", "zhao", "lu", "zhang", "carlos", "jorge", "itoi", "daiki", "jaime", "li", \
    "ayana", "luqi", "hailing", "kexin", "goto", "reamey", "yingqing", "tiffnay", "hanieh", "katya",
]
four_hour_speakers = [1, 2, 3, 4, 6, 7, 8, 9, 11, 21]
one_hour_speakers = [5, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30]
split_rule_english = {
    # 4h speakers x 10
    "four_hour_speakers":{
        # 48+40+100=188mins each
        "train": [
            "0_9_9", "0_10_10", "0_11_11", "0_12_12", "0_13_13", "0_14_14", "0_15_15", "0_16_16", \
            "0_17_17", "0_18_18", "0_19_19", "0_20_20", "0_21_21", "0_22_22", "0_23_23", "0_24_24", \
            "0_25_25", "0_26_26", "0_27_27", "0_28_28", "0_29_29", "0_30_30", "0_31_31", "0_32_32", \
            "0_33_33", "0_34_34", "0_35_35", "0_36_36", "0_37_37", "0_38_38", "0_39_39", "0_40_40", \
            "0_41_41", "0_42_42", "0_43_43", "0_44_44", "0_45_45", "0_46_46", "0_47_47", "0_48_48", \
            "0_49_49", "0_50_50", "0_51_51", "0_52_52", "0_53_53", "0_54_54", "0_55_55", "0_56_56", \
            
            "0_66_66", "0_67_67", "0_68_68", "0_69_69", "0_70_70", "0_71_71",  \
            "0_74_74", "0_75_75", "0_76_76", "0_77_77", "0_78_78", "0_79_79",  \
            "0_82_82", "0_83_83", "0_84_84", "0_85_85",  \
            "0_88_88", "0_89_89", "0_90_90", "0_91_91", "0_92_92", "0_93_93",  \
            "0_96_96", "0_97_97", "0_98_98", "0_99_99", "0_100_100", "0_101_101",  \
            "0_104_104", "0_105_105", "0_106_106", "0_107_107", "0_108_108", "0_109_109",  \
            "0_112_112", "0_113_113", "0_114_114", "0_115_115", "0_116_116", "0_117_117",  \
            
            "1_2_2", "1_3_3", "1_4_4", "1_5_5", "1_6_6", "1_7_7", "1_8_8", "1_9_9", "1_10_10", "1_11_11",
        ],
        # 8+7+10=25mins each
        "val": [
            "0_57_57", "0_58_58", "0_59_59", "0_60_60", "0_61_61", "0_62_62", "0_63_63", "0_64_64", \
            "0_72_72", "0_80_80", "0_86_86", "0_94_94", "0_102_102", "0_110_110", "0_118_118", \
            "1_12_12",
        ],
        # 8+7+10=25mins each
        "test": [
           "0_1_1", "0_2_2", "0_3_3", "0_4_4", "0_5_5", "0_6_6", "0_7_7", "0_8_8", \
           "0_65_65", "0_73_73", "0_81_81", "0_87_87", "0_95_95", "0_103_103", "0_111_111", \
           "1_1_1",
        ],
    },
    
    # 1h speakers x 20
    "one_hour_speakers":{
        # 8+7+20=35mins each
        "train": [
            "0_9_9", "0_10_10", "0_11_11", "0_12_12", "0_13_13", "0_14_14", "0_15_15", "0_16_16", \
            "0_66_66", "0_74_74", "0_82_82", "0_88_88", "0_96_96", "0_104_104", "0_112_112", "0_118_118", \
            "1_2_2", "1_3_3", 
            "1_0_0", "1_4_4", # for speaker 29 only
        ],
        # 4+3.5+5 = 12.5mins each
        # 0_65_a and 0_65_b denote the frist and second half of sequence 0_65_65
        "val": [
            "0_5_5", "0_6_6", "0_7_7", "0_8_8",  \
            "0_65_b", "0_73_b", "0_81_b", "0_87_b", "0_95_b", "0_103_b", "0_111_b", \
            "1_1_b",
        ],
        # 4+3.5+5 = 12.5mins each
        "test": [
           "0_1_1", "0_2_2", "0_3_3", "0_4_4", \
           "0_65_a", "0_73_a", "0_81_a", "0_87_a", "0_95_a", "0_103_a", "0_111_a", \
           "1_1_a",
        ],
    },
}

In [7]:
def overwrite_move(src, dst):
    shutil.copy2(src, dst)
    os.remove(src)
def cut_sequence(source_path, save_path_a, save_path_b, file_id, fps = 15, sr = 16000, tmp="./datasets/beat_tmp/"):
    if not os.path.exists(tmp): os.mkdir(tmp)
    cut_point = 30 if file_id.split("_")[0] == "0" else 300 #in seconds
    if source_path.endswith(".npy"):
        data = np.load(source_path)
        data_a = data[:sr*cut_point]
        data_b = data[sr*cut_point:]
        np.save(save_path_a, data_a)
        np.save(save_path_b, data_b)
        
    elif source_path.endswith(".bvh"):
        copy_lines = 431 if "full" in source_path or "vis" in source_path else 0
        with open(source_path, "r") as data:
            with open(save_path_a, "w") as data_a:
                with open(save_path_b, "w") as data_b:
                    for i, line_data in enumerate(data.readlines()):
                        if i < copy_lines:
                            data_a.write(line_data)
                            data_b.write(line_data)
                        elif i < cut_point * fps:
                            data_a.write(line_data)
                        else:
                            data_b.write(line_data)
    
    elif source_path.endswith(".json"):
        with open(source_path, "r", encoding='utf-8') as data:
            json_file = json.load(data)
            with open(save_path_a, "w") as data_a:
                with open(save_path_b, "w") as data_b:
                    new_frames_a = []
                    new_frames_b = []
                    for json_data in json_file["frames"]:
                        if json_data["time"] < cut_point:
                            new_frames_a.append(json_data)
                        else:
                            new_frame = json_data.copy()
                            new_frame["time"]-=cut_point
                            new_frames_b.append(new_frame)
                    json_new_a = {"names":json_file["names"], "frames": new_frames_a}
                    json_new_b = {"names":json_file["names"], "frames": new_frames_b}
                    json.dump(json_new_a, data_a)
                    json.dump(json_new_b, data_b) 
        
    else:
        # processing in the dataloader
        shutil.copy(source_path, save_path_a)
        shutil.copy(source_path, save_path_b)
    try:
        overwrite_move(source_path, tmp)
    except Exception as e:
        print(f"({e}) in move: {source_path}")

In [12]:
# spilt data
folders = os.listdir(default_path)
if not os.path.exists(default_path.replace("train", "val")): os.mkdir(default_path.replace("train", "val"))
if not os.path.exists(default_path.replace("train", "test")): os.mkdir(default_path.replace("train", "test"))
endwith = []
for folder in folders:
    if not os.path.exists(default_path.replace("train", "val")+"/"+folder): os.mkdir(default_path.replace("train", "val")+"/"+folder)
    if not os.path.exists(default_path.replace("train", "test")+"/"+folder): os.mkdir(default_path.replace("train", "test")+"/"+folder)
    end = os.listdir(default_path+"/"+folder)[-1].split(".")[-1]
    endwith.append(end)
    
for speaker_id in tqdm(speakers):
    speaker_type = "one_hour_speakers" if str(speaker_id) in one_hour_speakers else "four_hour_speakers"
    val = split_rule_english[speaker_type]["val"]
    test = split_rule_english[speaker_type]["test"]
    for file_id in val:
        print(f"file_id: {file_id}")
        for ide, folder in enumerate(folders):
            if "b" in file_id:
                cut_sequence(
                    source_path=f"{default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id.split('_')[0]}_{file_id.split('_')[1]}_{file_id.split('_')[1]}.{endwith[ide]}",
                    save_path_a=f"{default_path.replace('train', 'test')}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id.split('_')[0]}_{file_id.split('_')[1]}_a.{endwith[ide]}",
                    save_path_b=f"{default_path.replace('train', 'val')}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id.split('_')[0]}_{file_id.split('_')[1]}_b.{endwith[ide]}",
                    file_id = file_id,
                        )
            else:
                #pass
                try:
                    print(f"moving to val: {default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id}.{endwith[ide]}")
                    overwrite_move(f"{default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id}.{endwith[ide]}", f"{default_path.replace('train', 'val')}/{folder}/")
                except Exception as e:
                    print(f"(val) ({e}) in move: {default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id}.{endwith[ide]}")
    for file_id in test:
        for ide, folder in enumerate(folders):
            if "a" in file_id:
                pass
            else:
                #pass
                try:
                    overwrite_move(f"{default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id}.{endwith[ide]}", f"{default_path.replace('train', 'test')}/{folder}/")
                
                except Exception as e:
                    print(f"(test) ({e}) in move: {default_path}/{folder}/{speaker_id}_{speaker_names[speaker_id-1]}_{file_id}.{endwith[ide]}")


  0%|          | 0/4 [00:00<?, ?it/s]

file_id: 0_57_57
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/2_scott_0_57_57.json
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/text/2_scott_0_57_57.TextGrid
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/wave16k/2_scott_0_57_57.npy
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/sem/2_scott_0_57_57.txt
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/emo/2_scott_0_57_57.csv
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_6d/2_scott_0_57_57.6d
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot/2_scott_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_vis/2_scott_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/head_rot/2_scott_0_57_57.6d
file_id: 0_58_58
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/2_scott_0_58_58.json
moving to val: ./datasets/beat_cache/beat_

 25%|██▌       | 1/4 [00:00<00:02,  1.09it/s]

file_id: 0_57_57
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/4_lawrence_0_57_57.json
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/text/4_lawrence_0_57_57.TextGrid
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/wave16k/4_lawrence_0_57_57.npy
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/sem/4_lawrence_0_57_57.txt
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/emo/4_lawrence_0_57_57.csv
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_6d/4_lawrence_0_57_57.6d
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot/4_lawrence_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_vis/4_lawrence_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/head_rot/4_lawrence_0_57_57.6d
file_id: 0_58_58
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/4_lawrence_0_58_58.json
moving to va

 50%|█████     | 2/4 [00:01<00:01,  1.02it/s]

file_id: 0_57_57
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/6_carla_0_57_57.json
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/text/6_carla_0_57_57.TextGrid
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/wave16k/6_carla_0_57_57.npy
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/sem/6_carla_0_57_57.txt
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/emo/6_carla_0_57_57.csv
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_6d/6_carla_0_57_57.6d
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot/6_carla_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_vis/6_carla_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/head_rot/6_carla_0_57_57.6d
file_id: 0_58_58
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/6_carla_0_58_58.json
moving to val: ./datasets/beat_cache/beat_

 75%|███████▌  | 3/4 [00:02<00:00,  1.31it/s]

file_id: 0_57_57
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/8_catherine_0_57_57.json
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/text/8_catherine_0_57_57.TextGrid
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/wave16k/8_catherine_0_57_57.npy
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/sem/8_catherine_0_57_57.txt
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/emo/8_catherine_0_57_57.csv
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_6d/8_catherine_0_57_57.6d
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot/8_catherine_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/bvh_rot_vis/8_catherine_0_57_57.bvh
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/head_rot/8_catherine_0_57_57.6d
file_id: 0_58_58
moving to val: ./datasets/beat_cache/beat_4english_15_141/train/facial52/8_catherine_0_58_58.json
mo

100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


In [12]:
import glob
import shutil
import os
pose_files = sorted(glob.glob("./datasets/beat_cache/beat_4english_15_141/test/wave16k/*.npy"), key=str,)
for pose_file in pose_files:
    pose_file = pose_file.split('/')[-1][:-4]
    pose_id = pose_file.split('_')[0]
    wav_file = "./datasets/beat_rawdata_english/" + pose_id + "/" + pose_file + ".wav"
    move_target_path = "../../DiffSHEG-main/audios/BEAT_2468speakers"
    overwrite_move(wav_file, move_target_path)


In [13]:
import numpy as np

target_list = joint_list["beat_141"]
def getBoneId(bone_name):
    index = 0
    for key, value in joint_list["beat_141"].items():
        if bone_name == key:
            return index
        index += 1
    #return joint_list["beat_141"].index(bone_name)  # 如果找不到返回-1，或者你可以抛出异常

def readBoneStructure(path):
    # 初始化一个空列表来存储父子关系对
    bone_pairs = []

    # 读取文件
    with open(path, 'r') as file:
        for line in file:
            # 去除行尾的换行符和空格，并分割成父子关节名
            parent, child = line.strip().split(',')
            # 使用getBoneId函数将关节名转换为ID
            parent_id = getBoneId(parent)
            child_id = getBoneId(child)
            # 如果转换成功（即ID不是-1），则添加到列表中
            if parent_id != -1 and child_id != -1:
                bone_pairs.append((parent_id, child_id))
            else:
                # 如果转换失败，可以选择打印错误消息或抛出异常
                print(f"Error: Unable to convert bone names to IDs for line: {line.strip()}")

    # 将列表转换为N*2的numpy数组
    bone_structure_array = np.array(bone_pairs)
    return bone_structure_array

print(readBoneStructure(f"../spine_neck_141_boneid_structure"))

[[ 0  1]
 [ 1  2]
 [ 0 25]
 [25 26]
 [26 27]
 [27 28]
 [28 29]
 [29 30]
 [30 31]
 [28 32]
 [32 33]
 [33 34]
 [34 35]
 [32 36]
 [36 37]
 [37 38]
 [38 39]
 [28 40]
 [40 41]
 [41 42]
 [42 43]
 [40 44]
 [44 45]
 [45 46]
 [ 0  3]
 [ 3  4]
 [ 4  5]
 [ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 6 10]
 [10 11]
 [11 12]
 [12 13]
 [10 14]
 [14 15]
 [15 16]
 [16 17]
 [ 6 18]
 [18 19]
 [19 20]
 [20 21]
 [18 22]
 [22 23]
 [23 24]]


In [11]:
index = 0
for key, value in joint_list["beat_141"].items():
    if 'LHandT3' == key:
        print(f"Found 'LHandT3' in list at {index}.")
    index += 1

Found 'LHandT3' in list at 46.
